C:\Users\sik2k\AppData\Local\Temp\ipykernel_4004\1841758922.py:11: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(DATA / "cleaned_master.csv", low_memory=False, parse_dates=["transaction_time"], infer_datetime_format=True)


ValueError: Missing column provided to 'parse_dates': 'transaction_time'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

DATA = Path("../data")
REPORT = Path("../report")
REPORT.mkdir(exist_ok=True)

# Load without forcing a date column
df = pd.read_csv(DATA / "cleaned_master.csv", low_memory=False)

# Try to detect/construct a datetime column, then standardize name -> transaction_time
date_candidates_ordered = [
    "transaction_time", "time_of_transactions", "transaction_date",
    "date", "datetime", "timestamp", "time"
]
dt_col = next((c for c in date_candidates_ordered if c in df.columns), None)

if dt_col:
    df["transaction_time"] = pd.to_datetime(df[dt_col], errors="coerce", infer_datetime_format=True)
else:
    # Heuristic: try to parse any object column that looks like a date
    dt_col = None
    for c in df.columns:
        if df[c].dtype == "object":
            s = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
            if s.notna().mean() > 0.5:  # parsed successfully for most rows
                df["transaction_time"] = s
                dt_col = c
                break

if "transaction_time" in df.columns:
    df["year"] = df["transaction_time"].dt.year
    df["month"] = df["transaction_time"].dt.to_period("M")
    df["week"] = df["transaction_time"].dt.isocalendar().week.astype("Int64")
else:
    print("No usable date column found; time-based analyses will be skipped.")


C:\Users\sik2k\AppData\Local\Temp\ipykernel_4004\2337203277.py:21: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["transaction_time"] = pd.to_datetime(df[dt_col], errors="coerce", infer_datetime_format=True)
